# 1. Load Packages

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('analysis-tools')
from analysis_tools.common.utils import *
from analysis_tools import utils, random, eda

import tensorflow as tf
utils.set_memory_growth()
random.set_random_seed_tf()

exp_name = "GRU"
class PATH:
    root   = abspath('.')
    input  = join(root, 'data')
    output = join(root, 'output')
    ckpt   = join(root, 'ckpt', exp_name)
    eda    = join(root, 'eda', exp_name)    

# 2. Load Dataset
- TurbID \
Wind turbine ID, 발전기 ID (1~134)
- Day \
Day of the record, 날짜 (1~222)
- Tmstamp(HH:MM) \
Created time of the record, 시간 (00:00 ~ 23:50)
- Wspd(m/s) \
The wind speed recorded by the anemometer, 풍속
- Wdir(°) \
wind direction, 터빈이 바라보는 각도와 실제 바람 방향 각도 차이
- Etmp(℃) \
Temperature of the surounding environment, 외부 온도
- Itmp(℃) \
Temperature inside the turbine nacelle, 터빈 내부 온도
- Ndir(°) \
Nacelle direction, i.e., the yaw angle of the nacelle, 터빈이 바라보는방향 각도
- Pab(°) \
Pitch angle of blade,터빈 당 3개의 날이 있으며 각각의 각도가 다름
- Prtv(kW) \
Reactive power, 무효전력 : 에너지원을 필요로 하지 않는 전력
- **Patv(kW) -> Target** \
Active power, 유효전력 : 실제로 터빈을 돌리는 일을 하는 전력

In [ ]:
data_raw  = pd.read_csv(join(PATH.input, 'train_data.csv'))

test_data_raw = data_raw.query("Day >= 244")
data_raw = data_raw.query("Day < 244")

cols_raw = data_raw.columns
target = 'Patv'
data_raw.info()
data_raw

# 3. EDA
1. Features vs Target(Patv)
    - TurbID, Day, Etmp, Itmp는 뚜렷한 관계가 보이지 않음
    - corr(Wspd, Patv) >> 0
        - 특정 값(1) 이하이면, Patv=0
    - Wdir: abs(Wdir)이 특정값(10) 이상이면, Patv=0
    - Ndir: 특정값(0, 180, 360)에서 Patv 높음
    - Pab1,2,3: 특정 값(20) 이상이면, Patv=0
        - 대략 0.03을 기준으로 층이 보인다.
    - Prtv: Patv와 선형적인 관계가 보인다.
        - Prtv<0의 경우와 Prtv>0의 경우, Patv와 선형적인 관계를 가지고 있기 때문에 Prtv_neg, Prtv_pos 로 분리하면 더 모델이 잘 이해할 수 있을 것 같다.
2. 전체적으로 이상치가 많다.
    - 특히, Patv=0 인 이상치들이 많아 보인다.
    - 이상치를 제거하면 더 관계를 잘 파악할 수 있을 것 같다.

In [ ]:
# eda.plot_features(data_raw)
# eda.plot_features_target(data_raw, target, alpha=0.01)

In [ ]:
# eda.plot_ts_features(data_raw.query("TurbID == 1").drop(columns=['TurbID']), title='TurbID=1', figsize=(30, 8))
# eda.plot_ts_features(data_raw.query("TurbID == 2").drop(columns=['TurbID']), title='TurbID=2', figsize=(30, 8))

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

tmp = data_raw.sample(100000)
output_notebook()

In [ ]:
graph = figure()
graph.scatter(tmp['Wdir'], tmp[target], alpha=0.1)
show(graph)

In [ ]:
tmp[tmp['Wdir'].abs() < 10][target].plot.hist()

In [ ]:
tmp[tmp['Wdir'].abs() > 10][target].plot.hist()

In [ ]:
# graph = figure()
# graph.scatter(tmp['Ndir'], tmp[target], alpha=0.1)
# show(graph)

In [ ]:
# graph = figure()
# graph.scatter(tmp['Pab1'], tmp[target], alpha=0.1)
# show(graph)

In [ ]:
# a = tmp['Pab1'].value_counts().sort_index()
# a[a.index > 0].head(50)

In [ ]:
# graph = figure()
# graph.scatter(tmp['Wspd'], tmp[target], alpha=0.1)
# show(graph)

In [ ]:
# %%time
# eda.plot_pair(data_raw, alpha=0.01)

---
## - Correlation Analysis

In [ ]:
# from statsmodels.graphics.tsaplots import plot_acf

# def plot_acfs(data, cols=None, lags=144*30+1):
#     if cols is None:
#         cols = data.columns
#     acfs = {}
#     fig, axes = plt.subplots(len(cols), figsize=(40, len(cols)*2))
#     for ax, col in zip(axes.flat, cols):
#         plot_acf(data[col], lags=lags, missing='drop', auto_ylims=True, ax=ax, title=col)
#         xticks      = np.arange(0, lags, 144)
#         xticklabels = [f"{xtick//144} {'day' if xtick<=144 else 'days'}" for xtick in xticks]
#         ax.set_xticks(xticks, xticklabels)
#         ax.set_ylim([-1, 1])
#         ax.set_xlim([-1, lags+1])

#         _, line2 = ax.lines
#         acf = line2.get_ydata()[144*2:]
#         sorted_acf = np.array(sorted(enumerate(acf), key=lambda x: abs(x[1]), reverse=True), dtype='float32')
#         sorted_acf[:, 0] += 144*2
#         acfs[col] = sorted_acf
        
#         acf, _ = acfs[col][0]
#         ax.axvline(acf, color='k')
#     for ax in axes.flat[:-1]:
#         ax.set_xticklabels([])
    
#     return acfs

In [ ]:
# acfs = plot_acfs(data_proc.query("TurbID == 10"), cols=['Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir', 'Pab1', 'Prtv', 'Patv'])
# acfs = plot_acfs(data_proc.query("TurbID == 20"), cols=['Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir', 'Pab1', 'Prtv', 'Patv'])
# acfs = plot_acfs(data_proc.query("TurbID == 30"), cols=['Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir', 'Pab1', 'Prtv', 'Patv'])

# 4. Preprocessing

---
## - Missing values
1. 전체 4283712개 row중 48605개 row(1.1%)가 nan값을 가지고 있음
2. Nan값의 특징
    - 특정 TurbID에 많음 (126)
    - 특정 Day에 많음 (66, 67, 201)
    - 특정 Tmstamp에 많음 (00:00)

In [ ]:
# eda.plot_missing_value(data_raw, figsize=(30, 5))

In [ ]:
# eda.plot_ts_features(data_raw.query("TurbID == 126").drop(columns=['TurbID']))
# eda.plot_ts_features(data_raw.query("TurbID == 127").drop(columns=['TurbID']))

In [ ]:
# data_notnull = data[data_raw.notnull()]
# data_null    = data[data_raw.isna().any(1)]

# eda.plot_features(data_raw, title="Full data")
# eda.plot_features(data_notnull, title="Not null data")
# eda.plot_features(data_null, title="Null data")

In [ ]:
SAVE = True
def save_data(data, name):
    if SAVE:
        data.to_csv(join(PATH.input, f'{name}.csv'), index=False)

def check_nan(data, name):
    """Print number of data and nan rows

    Parameters
    ----------
    data : pandas.DataFrame
        Input data
    name : str
        Name to identify data
    """
    print("* Data name:", name)
    print("  - Number of rows:", len(data))
    print("  - Number of nan rows:", sum(data.isna().sum(axis='columns') > 0))
   
def compare_data(data1, data2, cols=None):
    if cols is None:
        cols = list(data1)
    eda.plot_features(data1[cols], data2[cols], figsize=(30, 8))
    eda.plot_pair(data1[cols], data2[cols], sample=1000, figsize=(15, 15))
    eda.plot_corr(data1[cols].corr(), data2[cols].corr(), figsize=(10, 10))
    check_nan(data1, "data1")
    check_nan(data2, "data2")
    
def get_efficiency(data):
    # Patv ≈ Efficiency * 0.5 * Wspd_cube * A * density
    # Efficiency < 0.6
    vals      = data['Wspd'].unique()
    min_val   = vals[vals > 0].min()
    Wspd      = np.maximum(data['Wspd'], min_val)
    Patv      = np.maximum(data['Patv'], 0)

    Wspd_cube = Wspd**3
    A         = np.pi * 82**2
    density   = 1.225
    return (Patv*1000) / (0.5*Wspd_cube * A * density)

---
## - Handle Errors
1. Patv값을 신뢰할 수 없는(Abnormal=1) row의 개수가 975144개(22.8%)로 적지 않다.
    - cond1: Patv <= 0 & Wspd > 2.5: 287362개(6.7%)
        - Patv <= 0 인 것이 딱히 문제되지는 않을 것 같다.
    - cond2: Pab1 > 89 | Pab2 > 89 | Pab3 > 89: 834589개(19.5%)
        - Pab의 최댓값이 99.98이고 20 이상의 값은 전부 0이기 때문에 0으로 처리
    - cond3: (Wdir < -180 | Wdir > 180) | (Ndir < -720 | Ndir > 720): 127(0.0%)
        - Pab와 마찬가지로 값이 특정값 이상이거나 이하인 경우 0인 경향이 크므로 0으로 처리
    - cond4: Patv is null: 48605개(1.1%)
        - 나중에 Patv를 modeling해서 채우자
2. Patv값을 신뢰할 수 없는 경우, 대부분의 값(whislo ~ whishi)의 Patv=0
3. Wdir이 180도를 넘어가거나 Ndir이 720도를 넘어가는 경우(cond3), 여전히 대부분의 값(whislo ~ whishi)은 Patv=0이나, 큰 Patv값이 나오고 Patv의 변화가 가장 컸다.

In [ ]:
def generate_datetime(data, start_date=datetime.datetime(2020, 12, 31)):
    data = copy(data)
    data['Day_tmp'] = (start_date + pd.to_timedelta(data['Day'], unit='day')).astype('string')
    data['Date'] = pd.to_datetime(data.apply(lambda row: f"{row['Day_tmp']} {row['Tmstamp']}:00", axis='columns'))
    for turbID in data['TurbID'].unique():
        d = data[data['TurbID'] == turbID]
        assert d['Date'].is_monotonic_increasing, "Data should be sorted"
        data.loc[data['TurbID'] == turbID, 'Time'] = 1 + np.arange(len(d))
    data.drop(columns=['Day_tmp'], inplace=True)
    data['Time'] = data['Time'].astype('int32')
    check_nan(data, "Generate datetime")
    return data.reset_index(drop=True)

def get_idxs_abnormal(data):
    cond1 = (data['Patv'] <= 0) & (data['Wspd'] > 2.5)
    cond2 = (data['Pab1'] > 89) | (data['Pab2'] > 89) | (data['Pab3'] > 89)
    cond3 = (data['Wdir'] < -180) | (data['Wdir'] > 180) | (data['Ndir'] < -720) | (data['Ndir'] > 720)
    cond4 = data['Patv'].isnull()
    cond  = cond1 | cond2 | cond3 | cond4
    return np.where(cond)[0], np.where(cond1)[0], np.where(cond2)[0], np.where(cond3)[0], np.where(cond4)[0]

def mark_abnormal_Patv(data):
    data = copy(data)
    data['Abnormal'] = 0
    data[[f'Abnormal_{i}' for i in [1, 2, 3, 4]]] = 0
    idxs_full, idxs_1, idxs_2, idxs_3, idxs_4 = get_idxs_abnormal(data)
    for num, i in enumerate([idxs_1, idxs_2, idxs_3, idxs_4], start=1):
        data.loc[data.iloc[i].index, f'Abnormal_{num}'] = 1
    data.loc[data.iloc[idxs_full].index, 'Abnormal'] = 1
    
    # Plot
    fig, axes = plt.subplots(5, figsize=(40, 8))
    for ax, col in zip(axes, ['Abnormal'] + [f'Abnormal_{i}' for i in [1, 2, 3, 4]]):
        sns.boxplot(x='Patv', y=col, data=data, orient='h', ax=ax);
    for ax in axes[:-1]:
        ax.set_xticklabels([])
        ax.set_xlabel(None)        

    # Adjust Patv
    data['Patv'] = data['Patv'].where(data['Abnormal_2'] == 0, 0)
    data['Patv'] = data['Patv'].where(data['Abnormal_3'] == 0, 0)

    data['Ndir'] = np.clip(data['Ndir'], -720, 720)
    data['Wdir'] = np.clip(data['Wdir'], -180, 180)
    for i in [1, 2, 3]:
        data[f'Pab{i}'] = np.clip(data[f'Pab{i}'], 0, 89)
    
    check_nan(data, "Mark anomaly Patv")
    return data

def manual_handling(data):
    def mark(data, col, min_val, max_val):
        data.loc[(data[col] < min_val) | (max_val < data[col]), col] = None

    data = copy(data)
    mark(data, 'Etmp', -20, 80)
    mark(data, 'Itmp', -10, 65)
    
    data.loc[(data['TurbID'] == 2) & (7533 <= data['Time']) & (data['Time'] <= 7899), 'Etmp']   = None
    data.loc[(data['TurbID'] == 2) & (24408 <= data['Time']) & (data['Time'] <= 24414), 'Etmp'] = None
    data.loc[(data['TurbID'] == 2) & (26562 <= data['Time']) & (data['Time'] <= 26575), 'Etmp'] = None
    
    check_nan(data, "Manual handling")
    return data

def outlier_handling(data, columns):
    @delayed
    def task(data_tid, window_size):
        for day in data_tid['Day'].unique():
            for column in columns:
                temp = data_tid.loc[(day <= data_tid['Day']) & (data_tid['Day'] <= day + window_size - 1), [column]]
                temp[f'{column}_diff'] = temp[column].diff(1)
                for col in (column, f'{column}_diff'):
                    stats = boxplot_stats(temp.dropna()[col])[0]
                    temp[col].where((stats['whislo'] <= temp[col]) & (temp[col] <= stats['whishi']), np.nan, inplace=True)
                temp.drop(columns=[f'{column}_diff'], inplace=True)
                data_tid.loc[temp.index, [column]] = temp.values
        return data_tid
    
    data = data.copy()
    tasks = [task(data[data['TurbID'] == turbID], window_size=2) for turbID in data['TurbID'].unique()]
    with ProgressBar():
        data_tids = compute(*tasks, scheduler='processes')
    
    for data_tid in tqdm(data_tids):
        data.loc[data_tid.index] = data_tid.values
    
    check_nan(data, "Outlier handling")
    return data

def find_neighbor(data, col, my_TurbID, plot=False):
    df = pd.DataFrame()
    for turbID in data['TurbID'].unique():
        df[turbID] = data[data['TurbID'] == turbID][col].values
    res = df - df[[my_TurbID]].values
    mae = res.abs().mean()
    rst = mae.drop(my_TurbID).idxmin()
    if plot:
        fig, (ax1, ax2) = plt.subplots(2, figsize=(40, 5))
        mae.plot.bar(ax=ax1)
        df[[my_TurbID, rst]].plot(ax=ax2)
    return rst

def get_dists_loc(input_dir):
    from sklearn.metrics.pairwise import euclidean_distances

    loc = pd.read_csv(join(input_dir, 'turb_location.csv')).set_index('TurbID')
    tids = loc.index
    dists_loc = np.full((len(loc)+1, len(loc)+1), np.inf)
    for i1 in range(len(loc)-1):
        for i2 in range(i1+1, len(loc)):
            t1, t2 = tids[i1], tids[i2]
            dists_loc[i1, i2] = euclidean_distances([loc.loc[t1].values], [loc.loc[t2].values])
    return dists_loc

def greedy_imputing(data, tids=None, k=10, verbose=False):
    def greedy_imputing_tid(data, tid_target, dists_loc, k, verbose):
        def affinity(tid):
            d        = data[data['TurbID'] == tid][cols].values
            d_target = data[data['TurbID'] == tid_target][cols].values
            diff = np.abs(d - d_target)
            return diff[~np.isnan(diff)].mean()
        
        cols       = ['Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir', 'Pab1', 'Pab2', 'Pab3', 'Prtv', 'Patv']
        neighbors  = [tid for tid, dist_loc in sorted(enumerate(dists_loc[tid_target], start=1), key=lambda x: x[1])[:k]]
        n_nan_prev = np.sum(data.loc[data['TurbID'] == tid_target, cols].isna().values)

        for tid in sorted(neighbors, key=affinity):
            a = data.loc[data['TurbID'] == tid_target, cols].values
            b = data.loc[data['TurbID'] == tid, cols].values
            data.loc[data['TurbID'] == tid_target, cols] = np.where(np.isnan(a) & (~np.isnan(b)), b, a)

        n_nan_after = np.sum(data[data['TurbID'] == tid_target].isna().values)
        if verbose and n_nan_after < n_nan_prev:
            print(f"[TurbID {tid_target}] Number of nan: {n_nan_prev} -> {n_nan_after}")
        return n_nan_after < n_nan_prev

    data = copy(data)
    dists_loc = get_dists_loc(PATH.input)

    if tids is None:
        tids = data['TurbID'].unique()

    while True:
        rsts = {}
        for tid in tqdm(tids):
            rsts[tid] = greedy_imputing_tid(data, tid, dists_loc, k, verbose)
        if not any(rsts.values()) or (k == 134):
            break

    check_nan(data, "Greedy imputing")
    return data

def impute_data(data, threshold=6 * 12, drop=False):
    data = copy(data)
    data_imp = pd.DataFrame()
    for turbID in tqdm(data['TurbID'].unique()):
        data_tid = data[data['TurbID'] == turbID]
        idxs = (data_tid.isna().sum(axis='columns') > 0)
        idxs_nan = idxs[idxs].index
        idxs_removed = []

        if len(idxs_nan) > 0:
            s, e = 0, 1
            while e < len(idxs_nan):
                cur = idxs_nan[s:e]
                if idxs_nan[e] == cur[-1] + 1:
                    e += 1
                else:
                    if len(cur) >= threshold:
                        idxs_removed += list(cur)
                    s = e
                    e = s + 1
            else:
                if len(cur) >= threshold:
                    idxs_removed += list(cur)

        data_tid_drop = data_tid.drop(idxs_removed)
        tmp = data.isna().sum()
        cols_nan = tmp[tmp > 0].index
        data_tid_drop[cols_nan] = data_tid_drop[cols_nan].interpolate().bfill().ffill()
        if drop:
            data_tid_imp = data_tid_drop
        else:
            data_tid_imp = data_tid
            data_tid_imp.loc[data_tid_drop.index] = data_tid_drop.values
        data_imp = data_imp.append(data_tid_imp)

    check_nan(data_imp, "Imputing")
    return data_imp.reset_index(drop=True)

In [ ]:
data1 = generate_datetime(data_raw)
save_data(data1, "data1")

data2 = mark_abnormal_Patv(data1)
save_data(data2, "data2")

data3 = manual_handling(data2)
save_data(data3, "data3")

data4 = outlier_handling(data3, ['Etmp', 'Itmp'])
save_data(data4, 'data4')

data5 = greedy_imputing(data4, tids=[2, 7, 8, 9, 13, 18, 24, 25, 26, 29, 30, 33, 36, 38, 40, 41, 50, 54, 60, 61, 65, 66, 67, 68, 79, 80, 82, 87, 88, 105, 118, 121, 122, 126, 129], k=20)
save_data(data5, 'data5')

data6 = impute_data(data5)
save_data(data6, 'data6')

In [ ]:
# tmp = data3
# eda.plot_two_features(tmp, 'Wspd', 'Patv', alpha=1)

In [ ]:
# eda.plot_features_target(data[cols_raw], target, alpha=0.01)
# eda.plot_features_target(data2[cols_raw], target, alpha=0.01)
# eda.plot_features_target(data3, target, alpha=0.01)

In [ ]:
# cols = cols_raw.drop(['TurbID', 'Pab2', 'Pab3'])

# eda.plot_ts_features(data3.query("TurbID == 1")[cols])
# eda.plot_ts_features(data3.query("TurbID == 2")[cols])
# eda.plot_ts_features(data3.query("TurbID == 67")[cols])

In [ ]:
# eda.plot_missing_value(data4[cols_raw])

In [ ]:
# cols = cols_raw.drop(['TurbID', 'Pab2', 'Pab3'])

# eda.plot_ts_features(data4.query("TurbID == 1")[cols])
# eda.plot_ts_features(data4.query("TurbID == 2")[cols])
# eda.plot_ts_features(data4.query("TurbID == 67")[cols])

In [ ]:
# eda.plot_features_target(data4[cols_raw], target, alpha=0.01)

In [ ]:
# data6 = pd.read_csv(join(PATH.input, 'data6.csv'))

In [ ]:
# eda.plot_missing_value(data6)
# eda.plot_features_target(data6, target, alpha=0.002)

In [ ]:
for tid in data6['TurbID'].unique():
    data = data6.query(f"TurbID == {tid}")
    data_nan = data[data.isna().any(1)]
    
    print("TurbID:", tid)
    display(data_nan.groupby("Day").count()['Time'].to_frame().T.style.background_gradient(axis=1))

In [ ]:
cols = ['Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir', 'Pab1', 'Prtv', 'Patv']
for tid in data6['TurbID'].unique():
    data = data6.query(f"TurbID == {tid}")[cols]
    eda.plot_ts_features(data, title=f"TurbID={tid}", figsize=(40, 8))

In [ ]:
data_imp = data6

## - Feature Engineering

In [ ]:
def feature_engineering(data):
    data = copy(data)
    
    # 1. EDA features
    data['Wspd_active']  = data['Wspd'] - 1
    data['Wspd_extreme'] = data['Wspd'] < 1
    data['Wspd_comb']    = data['Wspd_extreme'] * data['Wspd_active']
    
    data['Wdir_active']  = data['Wdir'].abs() - 10
    data['Wdir_extreme'] = data['Wdir'].abs() > 10
    data['Wdir_comb']    = data['Wdir_extreme'] * data['Wdir_active']
    
    Ndir_rad             = data['Ndir']
    data['Ndir_180']     = data['Ndir'].map(lambda x: np.min([abs(x-0), abs(x-180), abs(x-360)]))  # -180은 안 됨
    data['Ndir_extreme'] = data['Ndir_180'] > 90
    data['Ndir_comb']    = data['Ndir_extreme'] * data['Ndir_180']

    data['Pab']          = (data['Pab1'] + data['Pab2'] + data['Pab3'])/3
    data['Pab_extreme1'] = data['Pab'] < 0.03
    data['Pab_extreme2'] = data['Pab'] > 20

    data['Prtv_pos']     = data['Prtv'] > 0
    data['Prtv_abs']     = data['Prtv'].abs()
    data['Prtv_comb']    = data['Prtv_pos'] * data['Prtv_abs']
    
    
    # 2. Time
    DAY = 6*24  # 10minute * 6 * 24hour
    Time_in_day     = data['Time'] * (2*np.pi) / DAY
    data['Day_cos'] = np.cos(Time_in_day)
    data['Day_sin'] = np.sin(Time_in_day)

    YEAR = 365*DAY
    Time_in_year     = data['Time'] * (2*np.pi) / YEAR
    data['Year_cos'] = np.cos(Time_in_year)
    data['Year_sin'] = np.sin(Time_in_year)
    data['Weekday']  = data['Day'] % 7
    
    # 3. Direction
    Wdir_rad         = np.radians(data['Wdir'])
    data['Wdir_cos'] = np.cos(Wdir_rad)
    data['Wdir_sin'] = np.sin(Wdir_rad)
    data['Wspd_cos'] = data['Wspd_active'] * np.cos(Wdir_rad)
    data['Wspd_sin'] = data['Wspd_active'] * np.sin(Wdir_rad)
    
    data['Ndir_cos'] = np.cos(Ndir_rad)
    data['Ndir_sin'] = np.sin(Ndir_rad)
    
    # 4. RPM
    ALPHA = 40
    Pab_rad     = np.radians(data['Pab']+ALPHA)
    data['TSR'] = 1 / np.tan(Pab_rad)
    data['RPM'] = data['Wspd_active'] * data['TSR']
    
    # 5. Patv
    data['Wspd_cube'] = data['Wspd_active']**3
    min_val = data[data['Patv'] > 0]['Patv'].min()
    data['Patv_pos']  = np.maximum(data['Patv'], min_val)
    data['Patan_abs'] = np.arctan(data['Prtv_abs'] / data['Patv_pos'])
    
    return data

def feature_engineering_lag(data, cols, lags):
    data = copy(data)
    for turbID in tqdm(data['TurbID'].unique()):
        data_tid = data[data['TurbID'] == turbID]
        for lag in lags:
            data.loc[data_tid.index, [f'{col}_LAG{lag}' for col in cols]] = data_tid[cols].shift(144*lag).values
#         for lag in [3]:
#             data.loc[data_tid.index, [f'{col}_MA{lag}' for col in cols]]  = data_tid[cols].rolling(144*lag).mean().values
    data = data[data['Day'] > max(lags)]
    check_nan(data, "Feature engineering2")
    return data

In [ ]:
data_fe = feature_engineering(data_imp)
# save_data(data_fe, "data_fe")
data_fe

In [ ]:
eda.plot_features_target(data_fe, target, alpha=0.01, figsize=(30, 30))

In [ ]:
data_fe_bfill = copy(data_fe)
data_nan = data_fe_bfill[data_fe_bfill.isna().any(1)]
idxs_nan = data_nan.index

tmp = data_nan.isna().sum() > 0
cols = tmp[tmp].index

data_fe_bfill.loc[data_fe_bfill['Abnormal'] == 1, cols] = None
data_fe_bfill.bfill(inplace=True)

data_fe_bfill.loc[idxs_nan, cols] = data_nan[cols]

In [ ]:
eda.plot_features_target(data_fe_bfill, target, alpha=0.01, figsize=(30, 30))

## - Feature Selection
1. 전부 사용하면 overfitting이 되기 십상이다.
2. 여러 번의 피드백을 통해 feature를 선택하는 것이 바람직하다.

In [ ]:
# cols_sel = ['TurbID', 'Day', 'Time']
# cols_sel += ['Wspd', 'Wspd_extreme', 'Wspd_cube',
#              'Wdir_extreme',
#              'Ndir_180', 'Ndir_extreme',
#              'Pab_extreme1', 'Pab_extreme2',
#              'Prtv', 'Prtv_pos', 'Patan_abs',
#              'Etmp', 'Itmp',
#              'RPM']
# cols_sel += [target]

In [ ]:
cols_sel = ['TurbID', 'Day', 'Time']
cols_sel += ['Etmp', 'Itmp', 'Prtv', 
             'Wspd', 'Wspd_extreme', 'Wspd_comb', 
             'Wdir_active', 'Wdir_extreme', 'Wdir_comb', 
             'Ndir_cos', 'Ndir_sin', 'Ndir_extreme', 'Ndir_comb', 
             'Pab', 'Pab_extreme1', 'Pab_extreme2', 
             'Prtv_pos', 'Prtv_abs', 'Prtv_comb', 
             'Wdir_cos', 'Wdir_sin', 'Wspd_cos', 'Wspd_sin', 
             'RPM', 'Wspd_cube', 
             'Patan_abs']
cols_sel += ['Day_cos', 'Day_sin', 'Year_cos', 'Year_sin']
cols_sel += [target]

In [ ]:
data_sel = data_fe[cols_sel]
# data_sel = data_fe_bfill[cols_sel]
data_sel

In [ ]:
data_sel = feature_engineering_lag(data_sel, data_sel.columns.drop(["TurbID", 'Day', 'Time']), lags=[5])
data_sel

# 4. Modeling

In [ ]:
cols_in = data_sel.columns.drop(['Day', 'Time'])
idx_cols_in = [i for i, col in enumerate(data_sel.columns) if col in cols_in]

cols_out = ['Wspd', 'Patv']  # ['Etmp', 'Itmp', 'Prtv', 'Wspd_active', 'Wdir', 'Ndir', 'Pab', 'Patv']
idx_cols_out = [i for i, col in enumerate(data_sel.columns) if col in cols_out]

print(cols_in)
print(cols_out)

In [ ]:
def print_shape(*args):
    print("* Shape of dataset")
    for arg in args:
        print(f"  - {arg.shape}")

def scale(data, scaler):
    if not isinstance(data, np.ndarray):
        data = np.array(data, dtype='float32')
    if len(data.shape) == 3:
        return scaler.transform(data.reshape(-1, data.shape[-1])).reshape(data.shape)
    elif len(data.shape) == 2:
        return scaler.transform(data)
    else:
        raise ValueError("len(data.shape) should be 2 or 3")
        
def inverse_scale(data, scaler):
    if len(data.shape) == 3:
        return scaler.inverse_transform(data.reshape(-1, data.shape[-1])).reshape(data.shape)
    elif len(data.shape) == 2:
        return scaler.inverse_transform(data)
    else:
        raise ValueError("len(data.shape) should be 2 or 3")

## - Data split 1

In [ ]:
from sklearn.model_selection import train_test_split
from analysis_tools import preprocessing


def sliding_window(df, in_seq_len, out_seq_len, stride):
    if len(df) == in_seq_len:
        # test set
        return [df.values], None
    elif len(df) > in_seq_len:
        # training, validation set
        df_in  = df.iloc[:-out_seq_len]
        df_out = df.iloc[in_seq_len:]
    else:
        raise ValueError("len(df) is too short")

    ins, outs = [], []
    for i in range(0, len(df) - in_seq_len - out_seq_len + 1, stride):
        in_sel  = df_in.iloc[i:i + in_seq_len]
        ins.append(in_sel.values)
        out_sel = df_out.iloc[i:i + out_seq_len]
        outs.append(out_sel.values)
    return np.array(ins), np.array(outs)


def split_dataframe(df, in_seq_len, out_seq_len, stride, val_size):
    df = df.astype('float32')

    # 1. Sliding windows 생성
    windows_in, windows_out = [], []
    for tid in df['TurbID'].unique():
        window_in, window_out = sliding_window(df[df['TurbID'] == tid], in_seq_len, out_seq_len, stride)
        windows_in.append(window_in);  windows_out.append(window_out)
    windows_in, windows_out = np.array(windows_in), np.array(windows_out)

    T, B, S, D = windows_in.shape
    if windows_out.ndim == 1:  # test set
        pass
    else:
        T, B_, S_, D = windows_out.shape

    if B == 1:  # if there is one sequence, then test set
        return windows_in.reshape(T, S, D)
    else:  # training, validation set
        idxs_in,  idxs_out  = train_test_split(range(len(windows_in[1])), test_size=val_size)
        train_in, train_out = [], []
        val_in,   val_out   = [], []

        for window_in, window_out in zip(windows_in, windows_out):
            for idxs, ins, outs in zip([idxs_in, idxs_out], [train_in, val_in], [train_out, val_out]):
                for idx in idxs:
                    in_sel  = window_in[idx]
                    out_sel = window_out[idx]
                    if (np.isnan(in_sel).any()) or (np.isnan(out_sel).any()):
                        continue
                    ins.append(in_sel)
                    outs.append(out_sel)
        train_in  = np.array(train_in)
        train_out = np.array(train_out)
        val_in    = np.array(val_in)
        val_out   = np.array(val_out)

        return train_in, val_in, train_out, val_out
    
    
input_days  = 2
IN_SEQ_LEN  = input_days * 144
OUT_SEQ_LEN = 2 * 144
STRIDE      = 2 * 144
VAL_SIZE    = 0.1

n_tids          = data_sel['TurbID'].nunique()
max_day         = data_sel['Day'].max()
train_full_data = data_sel[data_sel['Day'] <= max_day-input_days]
test_data       = data_sel[max_day-input_days < data_sel['Day']]

train_in, val_in, train_out, val_out = split_dataframe(train_full_data, IN_SEQ_LEN, OUT_SEQ_LEN, STRIDE, VAL_SIZE)
test_in  = split_dataframe(test_data, IN_SEQ_LEN, OUT_SEQ_LEN, STRIDE, VAL_SIZE)
test_out = np.array([test_data_raw.query(f"TurbID == {tid}")[cols_out].values for tid in test_data_raw['TurbID'].unique()], dtype='float32')

train_in  = train_in[..., idx_cols_in]
train_out = train_out[..., idx_cols_out]

val_in    = val_in[..., idx_cols_in]
val_out   = val_out[..., idx_cols_out]

test_in   = test_in[..., idx_cols_in]

print_shape(train_in, train_out, val_in, val_out, test_in, test_out)

input_shape  = train_in.shape[1:]
output_shape = train_out.shape[1:]
print("input_shape:", input_shape, "output_shape:", output_shape)

## - Data split 2

In [ ]:
from sklearn.model_selection import train_test_split
from analysis_tools import preprocessing


def drop_missing_data(X, y):
    X_new, y_new = [], []
    for X_s, y_s in zip(X, y):
        if (np.isnan(X_s).sum() == 0) and (np.isnan(y_s).sum() == 0):
            X_new.append(X_s)
            y_new.append(y_s)
    return np.array(X_new), np.array(y_new)

def sliding_window(df, in_seq_len, out_seq_len, stride):
    X_df = df.iloc[:-out_seq_len]
    y_df = df.iloc[in_seq_len:]

    Xs = []
    ys = []
    for i in range(0, len(df) - in_seq_len - out_seq_len + 1, stride):
        X = X_df.iloc[i:i + in_seq_len]
        y = y_df.iloc[i:i + out_seq_len]
        if (X.isnull().values.any()) or (y.isnull().values.any()):
            continue
        Xs.append(X.values)
        ys.append(y.values)
    return np.array(Xs), np.array(ys)

def Xy_from_dataframe(df, in_seq_len, out_seq_len, stride):
    df = df.astype('float32')
    if len(df) == in_seq_len:  # test dataframe
        X = [df.values]  # expand_dim
        y = None
        X, _ = drop_missing_data(X, X)
    else:
        X, y = sliding_window(df, in_seq_len, out_seq_len, stride)
        X, y = drop_missing_data(X, y)
    return X, y

cols   = data_sel.columns.drop(['Day', 'Time'])
cols_X = data_sel.columns.drop(['Day', 'Time'])
cols_y = ['Wspd', 'Patv']  # ['Etmp', 'Itmp', 'Prtv', 'Wspd_active', 'Wdir', 'Ndir', 'Pab', 'Patv']
idx_cols_y = [i for i, col in enumerate(cols_X) if col in cols_y]

IN_SEQ_LEN  = 2*144
OUT_SEQ_LEN = 2*144
STRIDE      = 2*144

n_tids = data_sel['TurbID'].nunique()
train_full_X, train_full_y = n_tids*[None], n_tids*[None]
test_X,       test_y       = n_tids*[None], n_tids*[None]

train_full_data = data_sel[data_sel['Day'] < 242]
test_data       = data_sel[242 <= data_sel['Day']]

for i, tid in tqdm(list(enumerate(data_sel['TurbID'].unique()))):
    d_train_full = train_full_data.loc[train_full_data['TurbID'] == tid, cols]
    d_test       = test_data.loc[test_data['TurbID'] == tid, cols]
    
    train_full_X[i], train_full_y[i] = Xy_from_dataframe(d_train_full, IN_SEQ_LEN, OUT_SEQ_LEN, STRIDE)
    test_X[i],       _               = Xy_from_dataframe(d_test, IN_SEQ_LEN, OUT_SEQ_LEN, STRIDE)
    test_y[i] = [test_data_raw.query(f"TurbID == {tid}")[['Wspd', 'Patv']].values]
    
train_full_X = np.concatenate(train_full_X)
train_full_y = np.concatenate(train_full_y)[..., idx_cols_y]

test_X = np.concatenate(test_X)
test_y = np.concatenate(test_y)

train_X, val_X, train_y, val_y = train_test_split(train_full_X, train_full_y, test_size=0.1)

print_shape(train_X, train_y, val_X, val_y, test_X, test_y)

## - Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler_in = MinMaxScaler()
scaler_in.fit(train_in.reshape(-1, train_in.shape[-1]))
train_in_scale = scale(train_in, scaler_in)
val_in_scale   = scale(val_in, scaler_in)
test_in_scale  = scale(test_in, scaler_in)

scaler_out = MinMaxScaler()
scaler_out.fit(train_out.reshape(-1, train_out.shape[-1]))
train_out_scale = scale(train_out, scaler_out)
val_out_scale   = scale(val_out, scaler_out)
test_out_scale  = scale(test_out, scaler_out)

In [ ]:
from analysis_tools import modeling

BATCH_SIZE  = 32
SHUFFLE     = True

train_ds = modeling.generate_dataset_tf(train_in_scale, train_out_scale, BATCH_SIZE, SHUFFLE)
val_ds   = modeling.generate_dataset_tf(val_in_scale, val_out_scale, BATCH_SIZE, shuffle=False)
test_ds  = modeling.generate_dataset_tf(test_in_scale, test_out_scale, BATCH_SIZE, shuffle=False)

## - Transformer
https://keras.io/examples/timeseries/timeseries_transformer_classification/

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_model_transformer(
    input_shape,
    output_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

#     x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(output_shape[-1])(x)
    return keras.Model(inputs, outputs)

## - GRU

In [ ]:
def build_model(input_shape, output_shape, units, n_blocks, dropout=None):
    S,  D  = input_shape
    S_, D_ = output_shape
    
    model = keras.Sequential(name="GRU-Model") # Model
    model.add(keras.Input(shape=(S, D), name='Input-Layer'))
#     model.add(layers.Bidirectional(layers.GRU(units, return_sequences=False, dropout=dropout)))
#     model.add(layers.RepeatVector(S_, name='Repeat-Vector-Layer'))
    for _ in range(n_blocks):
        model.add(layers.Bidirectional(layers.GRU(units, return_sequences=True, dropout=dropout)))
    model.add(layers.Dense(units=D_))
    
    return model

# def build_model_GRU(input_shape, output_shape, units=256, dropout=None):
#     S,  D  = input_shape
#     S_, D_ = output_shape
#     model = keras.Sequential(name="GRU-Model") # Model
#     model.add(keras.Input(shape=(S, D), name='Input-Layer'))
#     model.add(layers.Bidirectional(layers.GRU(units=units, activation='tanh', recurrent_activation='sigmoid', stateful=False, dropout=dropout), name='Hidden-GRU-Encoder-Layer'))
#     model.add(layers.RepeatVector(S_, name='Repeat-Vector-Layer'))
#     model.add(layers.Bidirectional(layers.GRU(units=units, activation='tanh', recurrent_activation='sigmoid', stateful=False, return_sequences=True, dropout=dropout), name='Hidden-GRU-Decoder-Layer'))
#     model.add(layers.TimeDistributed(layers.Dense(units=D_), name='Output-Layer'))
#     return model

In [ ]:
from tensorflow.keras import losses, metrics
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from livelossplot import PlotLossesKeras
    
class Loss(losses.Loss):
    def __init__(self, loss_fn, **kwargs):
        super().__init__(**kwargs)
        self.loss_fn = loss_fn
    def call(self, y_true, y_pred):
        _, S, D = y_true.shape
        res = y_true - y_pred
        if self.loss_fn == 'rmse':
            return tf.sqrt(tf.reduce_mean(tf.square(res)))
        elif self.loss_fn == 'mse':
            return tf.reduce_mean(tf.square(res))
        elif self.loss_fn == 'mae':
            return tf.reduce_mean(tf.abs(res))
        else:
            raise NotImplementedError
            
            
def compile_and_fit(model, train_ds, val_ds, epochs, patience_es=10, patience_lr=3):
    model.compile('nadam', loss=Loss('rmse'), metrics=['mae'])
    os.makedirs(PATH.ckpt, exist_ok=True)
    return model.fit(train_ds, validation_data=val_ds,
                    epochs=epochs,
                    callbacks=[
                        PlotLossesKeras(),
                        EarlyStopping(patience=patience_es, restore_best_weights=True),
                        ReduceLROnPlateau(patience=patience_lr),
                        ModelCheckpoint(join(PATH.ckpt, '[{epoch:03d} epoch].h5'), save_best_only=False, save_weights_only=True),
                    ])

In [ ]:
# model = build_model_transformer(input_shape, output_shape, head_size=8, num_heads=32, ff_dim=16, num_transformer_blocks=4, mlp_units=[64], dropout=0.3, mlp_dropout=0.3)
model = build_model(input_shape, output_shape, units=128, n_blocks=2, dropout=0.1)
model.summary()

In [ ]:
compile_and_fit(model, train_ds, val_ds, epochs=1000, patience_es=15, patience_lr=3)

In [ ]:
# model.load_weights(join(PATH.ckpt, "[030 epoch].h5"))

# 5. Result

## - Training, validation set

In [ ]:
plt.style.use('ggplot')

In [ ]:
def plot_result(model, scaler_in, scaler_out, cols_in, cols_out, datasets, n_cols=5, figsize=(40, 20)):
    def metric(y, p):
        res = y-p
        res = res[~np.isnan(res)]
        return (np.abs(res).mean() + np.sqrt(((res)**2).mean()))/2
    
    for name, ds in datasets.items():
        outs_scale = []
        p_outs_scale = []
        for in_scale, out_scale in ds.as_numpy_iterator():
            p_out_scale = model.predict(in_scale, verbose=0)
            outs_scale.append(out_scale);  p_outs_scale.append(p_out_scale)
        
        outs_scale = np.concatenate(outs_scale)
        p_outs_scale = np.concatenate(p_outs_scale)

        outs = inverse_scale(outs_scale, scaler_out)[..., -1].reshape(-1)
        p_outs = inverse_scale(p_outs_scale, scaler_out)[..., -1].reshape(-1)
        print(f"[{name}] mean(MAE, RMSE): {metric(outs, p_outs):.2f}")
        
        for inp, out in ds.take(1):
            inp   = inp[:n_cols]
            out   = out[:n_cols]
            p_out = model.predict(inp, verbose=0)
            
        inps   = [pd.DataFrame(inverse_scale(d, scaler_in), columns=cols_in, dtype='float32') for d in inp]
        outs   = [pd.DataFrame(inverse_scale(d, scaler_out), columns=cols_out) for d in out]
        p_outs = [pd.DataFrame(inverse_scale(d, scaler_out), columns=cols_out) for d in p_out]
        
        n_rows = out.shape[-1]
        fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
        for ax_col, col in zip(axes, cols_out):
            for idx_row, (ax, out, p_out) in enumerate(zip(ax_col, outs, p_outs)):
                ax.plot(out[col], label='true')
                ax.plot(p_out[col], label=f'pred ({np.mean(abs(p_out[col]-out[col])):.3f})')
                ax.fill_between(range(len(out[col])), out[col], p_out[col], alpha=0.1, color='b')
                ax.set_xticklabels([])
                ax.legend()
                if idx_row == 0:
                    ax.set_ylabel(col)
        fig.tight_layout()
        plt.show()

In [ ]:
plot_result(model, scaler_in, scaler_out, cols_in, cols_out, {'train': train_ds, 'val': val_ds}, figsize=(40, 10))

## - Test set

In [ ]:
plot_result(model, scaler_in, scaler_out, cols_in, cols_out, {'test': test_ds}, figsize=(40, 10))